# 🤖 麻雀AI 学習ランチャー for Google Colab

このノートブックは、麻雀AIの**事前学習**と**強化学習**をGoogle Colab上で実行するためのものです。
セルを上から順番に実行してください。

## ステップ 0: 環境セットアップ

最初に、ハードウェアアクセラレータを**GPU**に設定し、Google Driveをマウントして、必要なライブラリをインストールします。

In [1]:
# GPUの確認 (メニュー > ランタイム > ランタイムのタイプを変更 で GPU を選択してください)
import tensorflow as tf
print(f"TensorFlow GPU Available: {len(tf.config.list_physical_devices('GPU')) > 0}")

TensorFlow GPU Available: False


In [1]:
# Google Driveのマウント
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
import os

# ★★★★★【重要】ご自身のプロジェクトパスに変更してください ★★★★★
PROJECT_PATH = '/content/drive/MyDrive/いろいろ/麻雀AI/googlecolab/MahjongEnhancedLeaning'
# ★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★

os.chdir(PROJECT_PATH)
print(f"Working directory changed to: {os.getcwd()}")

Working directory changed to: /content/drive/MyDrive/いろいろ/麻雀AI/googlecolab/MahjongEnhancedLeaning


### ライブラリのインストール

Colab環境とライブラリの依存関係を解決するため、インストールを2段階に分けて実行します。

In [4]:
#【根本対策 ①】tf-agentsの依存関係を手動で解決します。
# シェルの特殊文字(<, >)が誤認識されないよう、パッケージ名をクォーテーションで囲みます。
!pip install 'pygame>=2.5.0' 'gym<=0.23.0' tensorflow-probability

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 624.4/624.4 kB 10.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for gym: filename=gym-0.23.0-py3-none-any.whl size=697742 sha256=a1f961c9d849d4cd5b9e7f41461567efd7cf328dfb386c0877c4e2314594011e
  Stored in directory: /root/.cache/pip/wheels/c1/d8/e3/7004c3b587864c6b8b5b61290283c40499fe7ba2b9c206ac3a
Successfully built gym
  Attempting uninstall: gym
    Found existing installation: gym 0.25.2
    Uninstalling gym-0.25.2:
      Successfully uninstalled gym-0.25.2


In [2]:
#【根本対策 ②】残りのライブラリと、tf-agents本体を「依存関係をチェックしない」オプション付きでインストールします。
!pip install -r requirements.txt --no-dependencies

---
## -ステップ 1: 事前学習 (Supervised Learning)-

天鳳のログデータを使って、Transformerモデルの初期学習を行います。
この処理は**非常に時間がかかります**。完了すると、`models/`ディレクトリに`.keras`モデルファイルが生成されます。

In [ ]:
# データ生成スクリプト (もしあれば)
# !python generate_data.py

# 事前学習の実行
# !python train_transformer.py

---
## ステップ 2: 強化学習 (Reinforcement Learning)

事前学習で作成したモデルをベースに、自己対戦による強化学習を開始します。

**【初回のみ】`ngrok`のセットアップ:**
1. [ngrok公式サイト](https://ngrok.com/)でアカウントを登録し、[認証トークン](https://dashboard.ngrok.com/get-started/your-authtoken)をコピーします。
2. このノートブックの左側にある**鍵アイコン (シークレット)**を開きます。
3. `NGROK_AUTH_TOKEN` という名前で新しいシークレットを作成し、値にコピーしたトークンを貼り付けてください。

In [2]:
import os
from google.colab import userdata

# Colabのシークレットからngrokのトークンを環境変数に設定
try:
    NGROK_TOKEN = userdata.get('NGROK_AUTH_TOKEN')
    os.environ['NGROK_AUTH_TOKEN'] = NGROK_TOKEN
    print("✅ NGROK_AUTH_TOKEN has been set.")
except userdata.SecretNotFoundError:
    print("⚠️ NGROK_AUTH_TOKEN secret not found. Please set it up.")

✅ NGROK_AUTH_TOKEN has been set.


In [3]:
# 強化学習を開始
!NGROK_AUTH_TOKEN=$NGROK_AUTH_TOKEN python train_rl_tf_agents.py

2025-10-04 11:28:56.472523: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759577336.489446   19326 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759577336.493870   19326 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1759577336.506167   19326 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1759577336.506230   19326 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1759577336.506233   19326 computation_placer.cc:177] computation placer alr

---
### ダッシュボードへの接続

上のセルを実行すると、ログの中に `ws://....ngrok.io` から始まる公開URLが出力されます。

1. そのURLをコピーします。
2. ローカルPCで `dashboard-colab.html` をブラウザで開きます。
3. コピーしたURLを入力欄に貼り付け、「接続」ボタンをクリックすると、学習状況がリアルタイムで表示されます。